# Market by Order (MBO) vs Market by Limit (MBL) - Python

## Overview
This sample is designed to illustrate the difference between **Market by Order (MBO)** and **Market by Limit (MBL)**.

### Order Book
The Order Book contains the list of orders (or quotes) to record the interest of buyers (bid side) and sellers (ask side) in a particular financial instrument.
Then, the matching engine (specific to each exchange or broker) uses the book to determine which orders can be fully or partially executed (trades). 

### Market by Order (MBO)
The **Market by Order (MBO)** is the simple view of the Order Book. It describes an order-based (or quote-based) data feed that provides the ability to view individual queue position, the full order book and the details of each individual order or quote at each price level.

The order or quote contains :
* TimeStamp : date and time of the order
* Identifier : unique identifier of the order
* Price : limit price to buy or sell
* Size : quantity to buy or sell
* Condition : trading condition (the order type : limit, market ...)
* Side : Bid or Ask

### Market by Limit (MBL)
**Market by Limit (MBL)**, also known as **Market by Price (MBP)**, is the price-based data of the book.
It is built using the MBO. It restricts updates for a given side (bid or ask) to a maximum of specified depth price levels and consolidates all the quantity (size) into a single level (or limit) for each price level, which includes the total quantity (size).

i.e. When several orders contain the same price, they are referred as a price level, meaning that if, for example, a bid comes at that price level, all the sell orders (ask) on that price level could potentially fulfill the order. 

Then the list of consolidated buy orders and sell orders are sorted : the highest bid price is on top of bids list and the lowest ask price is on top of the asks list.
The highest bid and the lowest ask are referred to as the **Top of the Book** (depth = 0). 

The level (or limit) contains :
* Depth : the index of price on the list
* Price : the price level
* Size : the total quantity
* Side : Bid or Ask

## Run the sample
### Inputs/outputs
The sample requires instrument's identifier, date time intervals as per inputs and returns the occured data information.

### Services used
This sample uses *gRPC requests* in order to retrieve MBO and MBL information from the hosted service. The queried endpoint in this script are:
* *TickQuotesService*: to directly retrieve MBO quotes data from the server.
* *TickTradesAndBookService*: to directly retrieve MBL levels data from the server.

### Packages required
1. Systemathics:
    * *systemathics.apis.services.tick.v1*
    * *systemathics.apis.type.shared.v1*
    *  *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    
***

# Run MBO vs MBL sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly

In [ ]:
pip install systemathics.apis --pre

In [ ]:
import os
import grpc
import csv
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import google.protobuf.wrappers_pb2 as wrappers
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.identifier_and_level_pb2 as identifier_and_level
import systemathics.apis.type.shared.v1.level_pb2 as level
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.type.shared.v1.book_updates_pb2 as book_updates
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2 as tick_trades_and_book
import systemathics.apis.services.tick.v1.tick_trades_and_book_pb2_grpc as tick_trades_and_book_service
import systemathics.apis.services.tick.v1.tick_quotes_pb2 as tick_quotes
import systemathics.apis.services.tick.v1.tick_quotes_pb2_grpc as tick_quotes_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Retrieve data
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
# Tha data is provided by ICE : let's use the ICE mapping codes to generate the identifier
# The ICE ticker
ticker = 'E:BNP'
provider = 'ICE'

# The source for full order book : Euronext
source = 'EQUITY_L2_973'

# Bokk depth
my_book_depth = 10

#### 3.2 Time period delimitation

In [ ]:
# Create time intervals (we are using Google date format)
# Full order book data avaialble (sample) : from 2021-11-01 to 2021-11-12
date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = 2021, month = 11, day = 1), 
    end_date = date.Date(year = 2021, month = 11, day = 1)
)

# Build the tick quotes request time interval (we are using Google date time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 12, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 12, minutes = 30, seconds = 0)
)

In [ ]:
# generate constraints based on the previous time selection
my_constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

#### 3.3 Request creation

##### 3.3.1 Market by Order (MBO)
The following code snippet create request for `quotes` data.

In [ ]:
# generate the quotes request
mbo_identifier = identifier.Identifier(exchange = source, ticker = ticker)
mbo_request = tick_quotes.TickQuotesRequest(
    identifiers = [mbo_identifier],
    constraints = my_constraints
)

##### 3.3.1 Market by Limit (MBL)
The following code snippet create request for `trades and book` data.

In [ ]:
# generate the tick trades request
mbl_identifier = identifier_and_level.IdentifierAndLevel(exchange = source, ticker = ticker, level = level.LEVEL_TRADES_AND_BOOK)
mbl_request = tick_trades_and_book.TickTradesAndBookRequest(
    identifiers = [mbl_identifier],
    constraints = my_constraints,
    book_updates = book_updates.BOOK_UPDATES_SNAPSHOTS_ONLY,
    max_depth = wrappers.Int32Value(value = my_book_depth),
    contributors = True
)

#### 3.4 Request processing


In the following code snippet, we request quotes for the given instrument.
The streamed response is exported on the fly in a file.

##### 3.4.1 Processing TickQuotes (MBO)

In [ ]:
import csv
from datetime import timedelta

quotes = []
quotes_count = 0
quotes_count_intern = 0

try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  

        # instantiate the tick quotes service
        mbo_service = tick_quotes_service.TickQuotesServiceStub(channel)

        # process the tick quotes request
        metadata = [('authorization', token)]
        for current_mbo in mbo_service.TickQuotes(request=mbo_request, metadata=metadata):

            # filter 1rst line = mapping (not used here since we are using only one instrument in request)
            if current_mbo.data.time_stamp.seconds > 0:
                quotes_count +=1
                
                            # store some quotes for the article:
                if quotes_count > 0 and quotes_count < 10:
                    quotes.append(current_mbo)

                time =datetime.fromtimestamp(current_mbo.data.time_stamp.seconds) + timedelta(microseconds = current_mbo.data.time_stamp.nanos/1000)
                for quote in current_mbo.data.quotes:
                    quotes_count_intern +=1
                    my_id = quote.id
                    my_side = quote.side
                    my_price= quote.price.value
                    my_size= quote.size.value
                    my_condition= quote.condition.value
                    
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())
    
print('Total quotes retrieved: {0}'.format(quotes_count))

##### 3.4.2 Processing TickTradesAndBook (MBL)

In [ ]:
try:
    
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick trades and book service
        mbl_service = tick_trades_and_book_service.TickTradesAndBookServiceStub(channel)
        
        # process the tick trades and book request
        time_stamps, trade_prices, trade_sizes, bid_prices, bid_sizes, ask_prices, ask_sizes  = [], [], [],[], [], [], []
        timestamp_count, limits_count, trades_count = 0, 0, 0

        for current_mbl in mbl_service.TickTradesAndBook(request = mbl_request, metadata = [('authorization', token)]):
            
            # get timestamps
            if current_mbl.data.time_stamp.seconds > 0:
                timestamp_count += 1
                time_stamps.append(current_mbl.data.time_stamp)
            
            # get book limits and set trade fields to nan
            if len(current_mbl.data.book.bid) > 0 or len(current_mbl.data.book.ask) > 0:
                bid_prices.append(current_mbl.data.book.bid[0].price)
                bid_sizes.append(current_mbl.data.book.bid[0].size)
                ask_prices.append(current_mbl.data.book.ask[0].price)
                ask_sizes.append(- current_mbl.data.book.ask[0].size)
                trade_prices.append(None)
                trade_sizes.append(None)
                limits_count += 1
            
            # get trades and set book limits fields to nan
            if current_mbl.data.trade.price > 0:
                trade_prices.append(current_mbl.data.trade.price)
                trade_sizes.append(current_mbl.data.trade.size)
                bid_prices.append(None)
                bid_sizes.append(None)
                ask_prices.append(None)
                ask_sizes.append(None)
                trades_count += 1
            
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())
    
print('Total time stamps retrieved: {0}'.format(timestamp_count))
print('Total trades retrieved: {0}'.format(trades_count))
print('Total book limits retrieved: {0}'.format(limits_count))    

### 4 Visualize results


#### 4.1 Market by Order (MBO)

The Market by order display type does not allow one to visualize the book completely for a given time stamp. The TickQuotes services only returns a series of quotes.
In the snippets below, we decide to visualize the first quotes returned in the response

In [ ]:
# small method to convert the side
def prettyprint(side):
    if side == 1:
        return "Bid"
    else:
        return 'Ask'

In [ ]:
quotes_id = [quotes[x].data.quotes[0].id for x in range(len(quotes))]
quotes_side = [prettyprint(quotes[x].data.quotes[0].side) for x in range(len(quotes))]
quotes_price = [quotes[x].data.quotes[0].price.value for x in range(len(quotes))]
quotes_size = [quotes[x].data.quotes[0].size.value for x in range(len(quotes))]
quotes_condition = [quotes[x].data.quotes[0].condition.value for x in range(len(quotes))]
time_stamps = [(datetime.fromtimestamp(quotes[x].data.time_stamp.seconds) + timedelta(microseconds = quotes[x].data.time_stamp.nanos/1000)) for x in range(len(quotes))]

# create a pandas dataframe with: dates, trades prices and sizes
d_mbo = {'TimeStamp': time_stamps, 'Id': quotes_id,'Side': quotes_side, 'Price': quotes_price, 'Size': quotes_size, 'Condition': quotes_condition}
df_mbo = pd.DataFrame(data=d_mbo)
df_mbo.to_csv('mbo_example_array.csv', index = False) # exporting if required

# Display quotes
df_mbo

From the list of quotes, it is possible to rebuild the full order book as we will see in the next part

#### 4.2 Market by Limit (MBL)

In the Market by Limit Book display type now appears the notion of depth. As the list of quotes are also returned for each limit, we can display the market for the selected depth.

##### 4.2.1 Visualize MBL in a dataframe

In [ ]:
# We will work on the last response
data = current_mbl.data

mbl_time_stamp = datetime.fromtimestamp(data.time_stamp.seconds) + timedelta(microseconds = data.time_stamp.nanos/1000)
book_max = max(len(data.book.bid), len(data.book.ask))
bid_prices = [data.book.bid[x].price for x in range(book_max)]
bid_sizes = [data.book.bid[x].size for x in range(book_max)]
bid_counts = [len(data.book.bid[x].quotes) for x in range(book_max)]
bid_details_joined = ['|'.join(['{}'.format(quote.size.value) for quote in current_mbl.data.book.bid[x].quotes]) for x in range(book_max)]
ask_prices = [data.book.ask[x].price for x in range(book_max)]
ask_sizes = [data.book.ask[x].size for x in range(book_max)]
ask_counts = [len(data.book.ask[x].quotes) for x in range(book_max)]
ask_details_joined = ['|'.join(['{}'.format(quote.size.value) for quote in current_mbl.data.book.ask[x].quotes]) for x in range(book_max)]

In [ ]:
# create a pandas dataframe with: dates, trades prices and sizes
d_mbl = {'Bid_Details': bid_details_joined,'Bid_#Quotes': bid_counts, 'Bid_Size': bid_sizes, 'Bid_Price': bid_prices, 'Ask_Price': ask_prices,  'Ask_Size': ask_sizes, 'Ask_#Quotes': ask_counts,'Ask_Details': ask_details_joined}
df_mbl = pd.DataFrame(data=d_mbl)
df_mbl.to_csv('mbl_example_array.csv', index = False) # exporting if required
df_mbl

##### 4.2.2 Visualize MBL in a bar chart
In a first step we have to define some functions to help us display the order sizes.

In [ ]:
# Gets the maximum size of list in a given list of list
# [[3,2,1,3,6],         
#  [4],         ====> returns 5
#  [5,12]],
def get_max_count(my_arrays):
    my_max = 0
    for array in my_arrays:
        if len(array) > my_max:
            my_max = len(array)
    return my_max



# Reverts an array of size i*various_size to an array of size max(various_size)*i where any new entry are zeros 
# [[3,2,1],          [[3,4,5],
#  [4],         ==>   [2,0,0],
#  [5,12]],           [5,12,0],
def reverse_details_array_and_fill(my_arrays):
    new_arrays = []
    my_max = get_max_count(my_arrays)
    for i in range(my_max):
        tmp = []
        for j in range(len(my_arrays)):
            if len(my_arrays[j]) < i+1:
                tmp.append(0)
            else:
                tmp.append(my_arrays[j][i])
        new_arrays.append(tmp)
    return new_arrays  

In [ ]:
# prepare data
bid_details = [[quote.size.value for quote in current_mbl.data.book.bid[x].quotes] for x in range(book_max)]
ask_details = [[quote.size.value for quote in current_mbl.data.book.ask[x].quotes] for x in range(book_max)]
bid_max_counts= get_max_count(bid_details)
ask_max_counts= get_max_count(ask_details)
print('Max order counts:')
print('--> Bid: {}'.format(bid_max_counts))
print('--> Ask: {}'.format(ask_max_counts))

In [ ]:
# reprocess the details of order counts in order to plot the result
reversed_bid_sizes = reverse_details_array_and_fill(bid_details)
reversed_ask_sizes = reverse_details_array_and_fill(ask_details)

In [ ]:
pip install colour

In [ ]:
from colour import Color

ask_start = Color("#e40000")
ask_end = Color("#f69b9b")
ask_colors = list(ask_start.range_to(ask_end,ask_max_counts))

bid_start = Color("#48b300")
bid_end = Color("#aff69b")
bid_colors = list(bid_start.range_to(bid_end,bid_max_counts))

We can now display the book by limit using `matplotlib` package

In [ ]:
import matplotlib.pyplot as plt

bid_colors = ['#1fff00','#1bd606','#19c808','#18ba0a','#15a30d','#13910f','#117f12','#0f6d15','#0d5e17','#0c5119','#0c5119', '#0c5119']
ask_colors = ['#88251e','#92221c','#9c1f19','#a51c17','#ac1a15','#b51713','#bf1411','#c8110f','#cf0f0d','#d90c0a','#e30907','#ea0705','#f30403','#ff0000']
fig,ax = plt.subplots(1,1, figsize=(17,8))
# bid
ax.bar(bid_prices, reversed_bid_sizes[0], color = bid_colors[0], edgecolor = 'black', width= 0.005, label = 'test')
for i_bid in range(1,bid_max_counts):
    ax.bar(bid_prices, reversed_bid_sizes[i_bid], bottom=reversed_bid_sizes[i_bid-1], color = bid_colors[i_bid], edgecolor = 'black', width= 0.005)

ax.set_xlabel("Price",fontsize=14)
ax.set_ylabel("Bid",color="green",fontsize=16)    
ax2 = ax.twinx()
ax2.bar(ask_prices, reversed_ask_sizes[0], color = ask_colors[0], edgecolor = 'black', width= 0.005)
for i_ask in range(1,ask_max_counts):
    ax2.bar(ask_prices, reversed_ask_sizes[i_ask], bottom=reversed_ask_sizes[i_ask-1], color = ask_colors[i_ask], edgecolor = 'black', width= 0.005)


ax2.set_ylabel("Ask",color="Red",fontsize=16)
    
# set the spacing between subplots
plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.2, hspace=0.8)
# add subtitle
plt.suptitle('Book by Limit for {0} : {1}'.format(ticker,mbl_time_stamp), size=20)
# plot
plt.plot()